<h1>Gender Classification of Facial Images Using CNN</h1>

<h2>This Notebook Covers</h2>
<h3><ol><li><a href="#1">Exploratory Data Analysis &amp; Data Cleaning</a></li>
    <li><a href="#2">Data Visualization</a></li>
    <li><a href="#3">Image Augmentation</a></li>
    <li><a href="#4">Model Development</a></li>
    <li><a href="#5">Model Evaluation</a></li>
    <li><a href="#6">Error Analysis</a></li>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from  IPython.display import display
from tensorflow.random import set_seed
np.random.seed(11)
set_seed(11)
random.seed(11)
!PYTHONHASHSEED=0

In [ ]:
df = pd.read_csv("../input/age-gender-and-ethnicity-face-data-csv/age_gender.csv")

<a id="1"></a><h2>Exploratory Data Analysis &amp; Data Cleaning</h2>

In [ ]:
df.shape

In [ ]:
df.head()

<h3>As seen above in the column 'pixels', the pixel data is in the form of a string where each pixel is separated by space. The function below converts the string into pixel array one row at a time.

In [ ]:
def img_arr(x):
    '''
    Function to convert pixel data (string) into array of pixels
    '''
    x=x.reset_index(drop=True)
    n = len(x) #number of rows
    for i in range(n):
        if i==0:
            arr = np.array(x[i].split()).astype(np.int16) #Initializing the array
        else:
            arr = np.append(arr,np.array(x[i].split()).astype(np.int16),axis=0) #Appending data to the array
    return arr.reshape(n,48,48,1) #reshaping the array to 4-dim image pixel array

In [ ]:
#Splitting dataset into X and y
X = df.iloc[:,4].copy()
y = df.iloc[:,2].copy()

In [ ]:
# As seen below the class is fairly balanced
y.value_counts()

In [ ]:
y.value_counts().plot(kind="bar")
plt.title("Label Distribution")
plt.xlabel("Labels")
plt.ylabel("Count");

In [ ]:
#splitting the data into train and te sets. 'te' set will be further split into validation and test sets 
X_train,X_te,y_train,y_te = train_test_split(X,y,test_size=0.3,random_state=11)

In [ ]:
#splitting 'te' set into validation and test set
X_val,X_test,y_val,y_test = train_test_split(X_te,y_te,test_size=0.15,random_state=11)

In [ ]:
#Converting the string of pixels into image array for each of train, val and test set
X_train = img_arr(X_train)
X_test = img_arr(X_test)
X_val = img_arr(X_val)

In [ ]:
y_train = y_train.values
y_test = y_test.values
y_val = y_val.values

<a id="2"></a><h2>Data Visualization</h2>
<h3>The code below displays 100 random faces and their genders. This helps in identifying anomalies in labeling and also helps in framing rules for image augmentation.

In [ ]:
rows=20 #rows in subplots
cols=5 #columns in subplots
samp = random.sample(range(X_train.shape[0]),rows*cols) #selecting 100 random samples
x_samp = X_train[samp,:,:,:]
y_samp = y_train[samp]

fig,ax = plt.subplots(rows,cols,figsize=(12,45))
r = 0
c = 0
for i in range(rows*cols):
    aa = x_samp[i,:,:,:].reshape(48,48)
    ax[r,c].axis("off")
    ax[r,c].imshow(aa,cmap="gray")
    ax[r,c].set_title(f"Gender: {'Female' if y_samp[i]==1 else 'Male'}")
    c+=1
    if c == cols:
        c=0
        r+=1
plt.show()

<h3>From the above data visualization, it is found that 0 indicates Male and 1 indicates Female. Also, there are a few images with wrong labels.

In [ ]:
set_seed(11)
random.seed(11)
np.random.seed(11)

<a id="3"></a><h2>Image Augmentation</h2>
<h3>Image augmentation is a process of transorming images with a set of pre-specified rules. Image augmentation doesn't result in additional images, rather it randomly transforms the images in every epoch and inputs to the CNN. This enables the CNN to train on multiple tranforms of the original image and prevents overfitting.</h3>
<h3>We must transform the training images only, validation and test images must be left untouched, except for scaling

In [ ]:
train_data_gen = ImageDataGenerator(rotation_range=30,
                                   width_shift_range=1,
                                    brightness_range=[0.8,1.2],
                                    zoom_range=[0.8,1.2],
                                    rescale=1/255
                                   )


val_data_gen = ImageDataGenerator(rescale=1/255)

test_data_gen = ImageDataGenerator(rescale=1/255)

<h3>The plot below shows random images in their original and augmented form

In [ ]:
fig,ax = plt.subplots(10,5,figsize=(15,25))
for n in range(10):    
    r = random.sample(range(X_train.shape[0]),1)[0]
    ax[n,0].imshow(X_train[r].reshape(48,48),cmap="gray")
    ax[n,0].set_title("Original")
    ax[n,0].axis("off")
    for i in range(1,5):
        ax[n,i].imshow(train_data_gen.random_transform(X_train[r]).reshape(48,48),cmap="gray")
        ax[n,i].set_title("Augmented")
        ax[n,i].axis("off")
plt.show()

In [ ]:
set_seed(11)
random.seed(11)
np.random.seed(11)
training_data = train_data_gen.flow(X_train,y_train,
                                   seed=11)

val_data = val_data_gen.flow(X_val,y_val,
                                   seed=11,shuffle=False)

test_data = test_data_gen.flow(X_test,y_test,
                                   seed=11,shuffle=False)

In [ ]:
INPUT_SHAPE = (48,48,1)

<a id="4"></a><h2>Model Development</h2>
The CNN below is inspired by VGG16 and to match the current data the network architecture is modified accordingly

In [ ]:
random.seed(11)
set_seed(11)
np.random.seed(11)
model = Sequential()

model.add(Conv2D(filters=64,kernel_size=3,strides=1,activation="relu",input_shape=INPUT_SHAPE))
model.add(Conv2D(filters=64,kernel_size=3,strides=1,activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding="same"))

model.add(Conv2D(filters=128,kernel_size=3,strides=1,activation="relu"))
model.add(Conv2D(filters=128,kernel_size=3,strides=1,activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),padding="same"))

model.add(Flatten())

model.add(Dense(units=512,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=1024,activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(units=1,activation="sigmoid"))

model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["binary_accuracy"])

In [ ]:
early_stop = EarlyStopping(monitor="val_loss",patience=5,mode="min") #Ensure the model doesn't overfit

In [ ]:
random.seed(11)
set_seed(11)
np.random.seed(11)
history = model.fit(training_data,batch_size=32,epochs=500,callbacks=early_stop,validation_data=val_data)

In [ ]:
#Dataframe capturing the accuracy and loss per epoch
loss_df = pd.DataFrame(history.history)
loss_df

In [ ]:
loss_df.plot();

<h3>Since, we got an idea on the optimum number of epochs to run from the above model training, now we'll concatenate the X_train, X_val and y_train, y_val to train the model on a larger dataset for a better performance. While training the model above I found 20 epochs as the optimum (might change due to randomness). Hence we'll train the final model for 20 epochs.

In [ ]:
Final_train = np.append(X_train,X_val,axis=0)
Final_val = np.append(y_train,y_val,axis=0)

In [ ]:
final_training_data = train_data_gen.flow(Final_train,Final_val,
                                   seed=11)

In [ ]:
random.seed(11)
set_seed(11)
np.random.seed(11)
final_model_history = model.fit(final_training_data,batch_size=32,epochs=20)

<a id="5"></a><h2>Model Evaluation

In [ ]:
model.evaluate(test_data)

In [ ]:
prediction = model.predict(test_data).flatten()

In [ ]:
print(prediction)

In [ ]:
prediction = np.round(prediction) #rounding so that the prediction >0.5 becones 1 and everything else becomes 0

In [ ]:
prediction

In [ ]:
sns.heatmap(confusion_matrix(y_test,prediction),annot=True,cbar=False,fmt="d")
plt.xlabel("Prediction")
plt.ylabel("Actual");

In [ ]:
print(classification_report(y_test,prediction))

<a id="6"></a><h2>Error Analysis</h2>
<h3>Analyzing the errors visually may help in tuning image augmentation parameters as well as the model architecture. It also gives an idea of how the model may perform in the future and determine if the model matches human level performance.

In [ ]:
error_index = (y_test != prediction)#finding error indices
y_test_error = y_test[error_index]
X_test_error = X_test[error_index]
prediction_error = prediction[error_index]

<h3>Below we visualize the errors and identify actual label vs predicted labels

In [ ]:
rows=int(np.floor(sum(error_index)/3)) #rows in subplots
cols=3 #columns in subplots
x_samp = X_test_error
y_samp = y_test_error

fig,ax = plt.subplots(rows,cols,figsize=(15,200))
r = 0
c = 0
for i in range((rows*cols)-1):
    aa = x_samp[i].reshape(48,48)
    ax[r,c].axis("off")
    ax[r,c].imshow(aa,cmap="gray")
    actual_lab = "Female" if y_samp[i]==1 else "Male"
    pred_lab = "Female" if int(prediction_error[i])==1 else "Male"
    ax[r,c].set_title(f'Actual: {actual_lab}\nPred: {pred_lab}')
    c+=1
    if c == cols:
        c=0
        r+=1
plt.show()

<h2>From the above error analysis, we can interpret that the model majorly misclassfied images of babies and kids (which even a human finds difficult to classify). This shows that beard and moustache and hair length might be the important features captured by the model for classifying the gender.